In [2]:
# from limes_x.agents import Ambassador
import os, sys
from pathlib import Path
from typing import Iterable
import yaml
from sshtunnel import SSHTunnelForwarder, create_logger
import requests
from typing import Any
from dataclasses import dataclass, field
import pickle, gzip, base64

from limes_x.models import Namespace, DataInstance, Endpoint, Transform, Solve, KeyGenerator
from limes_x.compute_module import ComputeModule
from limes_x.outpost.models import Message, Context, Config as RemoteConfig

In [3]:
class Connection:
    def __init__(self, host: str, config: RemoteConfig) -> None:
        self.server = SSHTunnelForwarder(
            host,
            remote_bind_address=(config.host, config.port),
            logger=create_logger(),
        )

        self.server.start()
        self._started = True

        self.compression_level = 3
        self.remote_config = config

        code, _ = self.Ping()
        assert code == 200, f"failed to connect to outpost [{host}]"
    
    def _send(self, context: Context, payload: Any=None):
        res = requests.post(
            f"http://localhost:{self.server.local_bind_port}/{self.remote_config.ver}",
            json=Message(
                context=context,
                payload=payload,
            ).Pack(),
        )

        msg = Message.Unpack(res.text)
        return res.status_code, msg

    def Ping(self, msg:str="hello"):
        return self._send(
            Context.PING,
            msg,
        )
    
    def ListTransforms(self) -> list[Transform]:
        code, msg = self._send(Context.LIST_TRANSFORMS)
        if code == 200:
            return [t for t in msg.payload if isinstance(t, Transform)]
        else:
            return []

    def Start(self):
        if not self._started: self.server.start()

    def Stop(self):
        self.server.stop()
        self._started = False

class Config:
    def __init__(self, path: Path) -> None:
        with open(path) as y:
            config = yaml.safe_load(y)

        self.connections: list[Connection] = []
        outposts = config.get("outposts", {}).get("ssh", [])
        for host in outposts:
            raw = outposts.get(host)
            if isinstance(raw, dict):
                remote_config = RemoteConfig.FromDict(raw)
            else: 
                remote_config = RemoteConfig()
            try:
                con = Connection(host, remote_config)
            except Exception as e:
                continue
                
            self.connections.append(con)

class Ambassador:
    def __init__(self, home: Path|str) -> None:
        home = Path(home)
        self.home = home
        if not home.exists():
            os.makedirs(home, exist_ok=True)
        else:
            assert home.is_dir(), f"home path [{home}] isn't folder"

        self.config = Config(home.joinpath("config.yml"))
        self.connections: list[Connection] = self.config.connections

    def __enter__(self):
        for con in self.connections:
            con.Start()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        for con in self.connections:
            con.Stop()

    def ListTransforms(self):
        for con in self.connections:
            x = con.ListTransforms()
            print(x)

    def GetPlan(self, given: Iterable[DataInstance], target: Transform):
        # given_endpoints = [i.endpoint for i in given]
        # plan = Solve(given_endpoints, target, [m.transform for m in self.modules])
        # return plan
        return

    def Execute(self, plan):
        pass

ns = Namespace()
a_proto = Endpoint(ns, {"a"})
a = DataInstance(a_proto, "/home/tony/workspace/python/Limes-all/Limes-x/test/outposts/local_outpost/workspace/a.txt")

target = Transform(ns)
target.AddRequirement({"b"})

with Ambassador("./test_ws") as lx:
    pass
    # for c in lx.connections:
    #     x = c.SetHome("/home/tony/workspace/python/Limes-all/Limes-x/test/outposts/local_outpost")
    #     print(x)
    lx.ListTransforms()
# plan = lx.GetPlan([a], target)
# print(plan)
# lx.Execute(plan)

2023-07-17 10:57:58,518| ERROR   | Secsh channel 0 open FAILED: Connection refused: Connect failed
2023-07-17 10:57:58,519| ERROR   | Could not establish connection from local ('127.0.0.1', 34193) to remote ('localhost', 12100) side of the tunnel: open new channel ssh error: ChannelException(2, 'Connect failed')
2023-07-17 10:57:59,666| ERROR   | Secsh channel 0 open FAILED: Connection refused: Connect failed
2023-07-17 10:57:59,667| ERROR   | Could not establish connection from local ('127.0.0.1', 44487) to remote ('localhost', 12100) side of the tunnel: open new channel ssh error: ChannelException(2, 'Connect failed')
